<a href="https://colab.research.google.com/github/MaCuur/AI_EGs/blob/main/hyperparameters_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error



In [2]:
# --- 1. Carga de Datos y Preprocesamiento (Reutilizado) ---
try:
    data = pd.read_csv("9002.csv", header=0).values
    data = data.astype(float).flatten()
    data = data[~np.isnan(data)]
    precipitation = pd.Series(data)

except Exception as e:
    print(f"Error al cargar o preprocesar el archivo CSV: {e}")
    print("Asegúrate de que el formato del archivo es el esperado.")
    exit()

dataset = precipitation.values.astype('float32').reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_dataset = scaler.fit_transform(dataset)
N_FEATURES = scaler.n_features_in_



In [3]:
# --- 2. Función de Preparación de Secuencias (Reutilizada) ---
def create_sequences(data, time_step, forecast_horizon):
    X, Y = [], []
    # Nota: Len(data) debe ser mayor que time_step + forecast_horizon - 1
    for i in range(len(data) - time_step - forecast_horizon + 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step + forecast_horizon - 1, 0])
    return np.array(X), np.array(Y)



In [4]:
# --- 3. Definición de Parámetros de Búsqueda ---

# Horizontes de interés
FORECAST_HORIZONS = [3, 5]

# Valores a probar para la Ventana de Entrada (Time Step)
TIME_STEPS_TO_TEST = [7, 15, 30, 60]

# Valores a probar para la Capacidad del Modelo (Unidades LSTM)
LSTM_UNITS_TO_TEST = [50, 100, 150]

# Variables para almacenar los mejores resultados
best_results = {h: {'mae': float('inf'), 'rmse': float('inf'), 'time_step': 0, 'units': 0} for h in FORECAST_HORIZONS}



In [5]:
# --- 4. Bucle Anidado para Grid Search (Búsqueda de Cuadrícula) ---

print("🚀 Iniciando Búsqueda de Hiperparámetros (Grid Search)...")

for HORIZON in FORECAST_HORIZONS:
    print(f"\n=======================================================")
    print(f"Búsqueda para Horizonte de Predicción: +{HORIZON} día(s)")
    print(f"=======================================================")

    # Prepara los datos una sola vez para el horizonte actual
    X, Y = create_sequences(scaled_dataset, max(TIME_STEPS_TO_TEST), HORIZON)

    # Dividir datos (80/20 secuencial)
    train_size = int(len(X) * 0.8)
    # Ya que el Time_Step varía, solo tomamos el array completo de X e Y y
    # la función create_sequences se encargará del padding/truncamiento de la secuencia

    for TIME_STEP in TIME_STEPS_TO_TEST:
        # Volver a crear X_train y X_test usando el TIME_STEP actual
        X_current, Y_current = create_sequences(scaled_dataset, TIME_STEP, HORIZON)

        # Volver a dividir con el nuevo tamaño de X, Y
        train_size_curr = int(len(X_current) * 0.8)
        X_train, X_test = X_current[:train_size_curr], X_current[train_size_curr:]
        Y_train, Y_test = Y_current[:train_size_curr], Y_current[train_size_curr:]

        # Reformar la entrada a [muestras, time_steps, características] (3D)
        X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

        for LSTM_UNITS in LSTM_UNITS_TO_TEST:
            print(f"  -> Probando: Time Step={TIME_STEP}, Unidades LSTM={LSTM_UNITS}")

            # 5. Construcción y Entrenamiento del Modelo
            model = Sequential()
            model.add(LSTM(LSTM_UNITS, activation='relu', input_shape=(TIME_STEP, 1)))
            model.add(Dense(1))

            model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

            early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', restore_best_weights=True)

            history = model.fit(
                X_train_reshaped, Y_train,
                epochs=100,
                batch_size=32,
                validation_data=(X_test_reshaped, Y_test),
                callbacks=[early_stop],
                verbose=0
            )

            # 6. Evaluación
            Y_pred_scaled = model.predict(X_test_reshaped, verbose=0)

            # Invertir el escalado (para MAE y RMSE en mm)
            dummy_pred = np.zeros((len(Y_pred_scaled), N_FEATURES))
            dummy_pred[:, 0] = Y_pred_scaled.flatten()
            Y_pred = scaler.inverse_transform(dummy_pred)[:, 0]

            dummy_test = np.zeros((len(Y_test), N_FEATURES))
            dummy_test[:, 0] = Y_test.flatten()
            Y_actual = scaler.inverse_transform(dummy_test)[:, 0]

            # Calcular métricas
            mae = mean_absolute_error(Y_actual, Y_pred)
            rmse = np.sqrt(mean_squared_error(Y_actual, Y_pred))

            print(f"     Resultado: MAE={mae:.4f} mm, RMSE={rmse:.4f} mm")

            # 7. Almacenar el mejor resultado
            if mae < best_results[HORIZON]['mae']:
                best_results[HORIZON]['mae'] = mae
                best_results[HORIZON]['rmse'] = rmse
                best_results[HORIZON]['time_step'] = TIME_STEP
                best_results[HORIZON]['units'] = LSTM_UNITS



🚀 Iniciando Búsqueda de Hiperparámetros (Grid Search)...

Búsqueda para Horizonte de Predicción: +3 día(s)
  -> Probando: Time Step=7, Unidades LSTM=50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


     Resultado: MAE=1.0300 mm, RMSE=2.6466 mm
  -> Probando: Time Step=7, Unidades LSTM=100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


     Resultado: MAE=0.9634 mm, RMSE=2.5911 mm
  -> Probando: Time Step=7, Unidades LSTM=150


...

     Resultado: MAE=0.9675 mm, RMSE=2.5602 mm
  -> Probando: Time Step=60, Unidades LSTM=100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


     Resultado: MAE=1.0540 mm, RMSE=2.5539 mm
  -> Probando: Time Step=60, Unidades LSTM=150


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


     Resultado: MAE=1.0337 mm, RMSE=2.5586 mm


In [6]:
# --- 8. Resumen de Resultados ---

print("\n\n✅ **RESUMEN DE LA OPTIMIZACIÓN DE HIPERPARÁMETROS**")
print("=====================================================")

for HORIZON in FORECAST_HORIZONS:
    best = best_results[HORIZON]
    print(f"\n### Resultados Óptimos para Predicción de +{HORIZON} Días")
    print(f"* **MAE Mínimo:** {best['mae']:.4f} mm")
    print(f"* **RMSE:** {best['rmse']:.4f} mm")
    print(f"* **Mejor TIME_STEP (Ventana de Entrada):** {best['time_step']} días")
    print(f"* **Mejores Unidades LSTM:** {best['units']}")
    print("-----------------------------------------------------")



✅ **RESUMEN DE LA OPTIMIZACIÓN DE HIPERPARÁMETROS**

### Resultados Óptimos para Predicción de +3 Días
* **MAE Mínimo:** 0.8896 mm
* **RMSE:** 2.5990 mm
* **Mejor TIME_STEP (Ventana de Entrada):** 7 días
* **Mejores Unidades LSTM:** 150
-----------------------------------------------------

### Resultados Óptimos para Predicción de +5 Días
* **MAE Mínimo:** 0.7805 mm
* **RMSE:** 2.5974 mm
* **Mejor TIME_STEP (Ventana de Entrada):** 15 días
* **Mejores Unidades LSTM:** 100
-----------------------------------------------------
